In [2]:
os.environ['USGS_USER'] = 'draperdl'
os.environ['USGS_PASSWORD'] = 'b8PrzH63Oa8O'

In [1]:
# %load datamanager/landsat5sr.py
import requests
import os
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor

usgs_host = 'https://espa.cr.usgs.gov/api/v0'
storage_directory = "/home/usgs/landsat5"

def default_auth():
    return (os.environ['USGS_USER'], os.environ['USGS_PASSWORD'])

def get_session(auth=None):
    if not hasattr(get_session,'s'):
        get_session.s = requests.Session()
        get_session.s.auth = auth if auth else default_auth()
    return get_session.s


def submit_order(list_of_ids):
    """Submit an order for the given list of landsat ids.
    For now, we are hardwiring all other parameters of the request (landsat5,
    sr, etc.)"""
    if len(list_of_ids) >= 5000:
        raise ValueError("Too many ids in a single request")
    # Build order body.  We aren't using any of the fancy options,
    # so this is quite simple
    order = {
        'tm5_collection': { 
            'inputs': list_of_ids,
            'products': ['sr']
        },
        'format': 'gtiff'
    }
    # send the order
    s = get_session()
    response = s.post(usgs_host+'/order',json=order)

    # error checking
    response.raise_for_status() # errors out if 404 etc.
    data = response.json()
    status = data['status'] if 'status' in data else 'unknown'
    if status != 'ordered':
        raise Exception('Order failed with status '+status)

    # return order id
    return data['orderid']


def get_order_status(id):
    s = get_session()
    response = s.get(usgs_host+'/order-status/'+id)
    response.raise_for_status()
    data = response.json()
    return data['status']

# copied directly from the epsg code... and wrong! (added 'ordered' status)
valid_statuses = ['ordered', 'complete', 'queued', 'oncache', 'onorder', 'purged',
                      'processing', 'error', 'unavailable', 'submitted']
all_but_purged = valid_statuses[:]
all_but_purged.remove('purged')

def get_open_orders():
    """Get the list and status of all orders that have not been purged"""
    s = get_session()
    response = s.get(usgs_host+'/list-orders', json={"status": all_but_purged })
    response.raise_for_status()
    return response.json()


def download_file(url,saveas):
    """Download a file, saving it in the path requested.  Overwrites existing file."""
    s = get_session()
    response = s.get(url,stream=True)
    with open(saveas,'wb') as out:
        for chunk in response.iter_content(chunk_size=65535):
            out.write(chunk)

def process_item(item):
    """Take the information about a single item (data file) and decide what to do about it:
    ignore it (if it is not ready or if we already have it) or download it (if it is new and ready)"""
    if item['status'] == 'complete':
        file = Path(storage_directory) / (item['name'] + '.tar.gz')
        if not file.exists():
            print('starting download of ' + item['name'])
            download_file(item['product_dload_url'],file)
            return
    print("xxx ignoring " + item['name'] + " xxx")


def download_available_results():
    """Download any products that are ready and have not previously been
    downloaded."""
    s = get_session()
    orders = get_open_orders()
    # usgs doesn't like it if you parallelize this loop, so we don't anymore
    for orderid in orders:
        response = s.get(usgs_host+'/item-status/'+orderid, json={"status": "complete"})
        response.raise_for_status()
        data = response.json()
        for item in data[orderid]:
            process_item(item)
    print("download complete")


In [3]:
s = get_session()

In [4]:
get_open_orders()

['espa-draperd@acm.org-05202019-024719-003']

In [11]:
get_order_status('espa-draperd@acm.org-0101811265895')

'complete'

In [4]:
response = s.get(usgs_host+'/item-status/'+'espa-draperd@acm.org-0101811265895')

In [5]:
response.json()

{'espa-draperd@acm.org-0101811265895': [{'cksum_download_url': 'https://edclpdsftp.cr.usgs.gov/orders/espa-draperd@acm.org-0101811265895/LT051950232006110901T2-SC20181126211541.md5',
   'completion_date': '2018-11-26 21:16:23.211489',
   'name': 'LT05_L1GS_195023_20061109_20180312_01_T2',
   'note': "''",
   'product_dload_url': 'https://edclpdsftp.cr.usgs.gov/orders/espa-draperd@acm.org-0101811265895/LT051950232006110901T2-SC20181126211541.tar.gz',
   'status': 'complete'}]}

In [7]:
process_item(response.json()['espa-draperd@acm.org-0101811265895'][0])

starting download of LT05_L1GS_195023_20061109_20180312_01_T2


In [6]:
!rm /home/usgs/landsat5/*

In [6]:
submit_order(["LT05_L1TP_168035_20061128_20180309_01_T1", "LT05_L1TP_168035_20061214_20180309_01_T1"])

{'tm5_collection': {'inputs': ['LT05_L1TP_168035_20061128_20180309_01_T1', 'LT05_L1TP_168035_20061214_20180309_01_T1'], 'products': ['sr']}, 'format': 'gtiff'}


'espa-draperd@acm.org-12012018-222824-456'

In [17]:
get_order_status('espa-draperd@acm.org-12022018-012026-667')

'ordered'

In [7]:
import numpy as np
import landsat5list
lst = np.array(landsat5list.potential_tiles)

In [8]:
np.random.shuffle(lst)
tofetch = list(lst[:10])
tofetch

['LT05_L1TP_189027_20060726_20161120_01_T1',
 'LT05_L1TP_197032_20060515_20180310_01_T1',
 'LT05_L1TP_176034_20060901_20161119_01_T1',
 'LT05_L1TP_201016_20060916_20161118_01_T1',
 'LT05_L1TP_175034_20060825_20161119_01_T1',
 'LT05_L1TP_182033_20060810_20180310_01_T1',
 'LT05_L1TP_176034_20060917_20161118_01_T1',
 'LT05_L1TP_173034_20060912_20161118_01_T1',
 'LT05_L1TP_190017_20060717_20161120_01_T1',
 'LT05_L1TP_169035_20061018_20161119_01_T1']

In [9]:
submit_order(list(tofetch))

'espa-draperd@acm.org-05202019-024719-003'

In [10]:
get_open_orders()

['espa-draperd@acm.org-05202019-024719-003']

In [21]:
get_order_status('espa-draperd@acm.org-12012018-222824-456')

'complete'

In [5]:
download_available_results()

starting download of LT05_L1TP_182033_20060810_20180310_01_T1
starting download of LT05_L1TP_169035_20061018_20161119_01_T1
starting download of LT05_L1TP_173034_20060912_20161118_01_T1
starting download of LT05_L1TP_201016_20060916_20161118_01_T1
starting download of LT05_L1TP_197032_20060515_20180310_01_T1
starting download of LT05_L1TP_176034_20060917_20161118_01_T1
starting download of LT05_L1TP_190017_20060717_20161120_01_T1
starting download of LT05_L1TP_176034_20060901_20161119_01_T1
starting download of LT05_L1TP_189027_20060726_20161120_01_T1
starting download of LT05_L1TP_175034_20060825_20161119_01_T1
download complete


In [10]:
!./ls5munge.sh /home/usgs/landsat5/LT05_L1TP_182033_20060810_20180310_01_T1.tar.gz

Working in temporary directory /tmp/tmp.afIVP5hEgT
Processing /home/usgs/landsat5/LT05_L1TP_182033_20060810_20180310_01_T1.tar.gz
./ls5munge.sh: line 20: gdalbuildvrt: command not found
./ls5munge.sh: line 25: gdal_translate: command not found
./ls5munge.sh: line 30: gdal_merge.py: command not found


In [11]:
# We have a problem here, because I have not found any way to activate the right conda environment from within a notebook.
# So for now, you have to open a terminal, activate fastai, then run ls5munge.sh manually.
# (TODO: It would be nice if we automatically determined which files needed to be munged...)